In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pylab as plt 
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from pythonds.basic.stack import Stack
from math import *
from sklearn.neighbors import KDTree
from lrg_plot_functions import *
from lrg_sum_functions import *
from cosmo_Calc import *
from divideByTwo import *
from read_data import *
from nearNeighbors import *
from scipy import stats
from astropy import stats
import healpy as hp
from localBKG_and_interlopersHEALPix import *
# import healpy as hpy
from makeCutoutTableFunc import *
from plotHistogramProjections3D import *
from read_bricks import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random
from sigSats import *
from sigSats_usingSumsat import *
from confidence_interval import *


In [3]:
# read in null data

# lowz
low_data_Nsat = np.load('/Users/mtownsend/Documents/LRG_Data/low_Nsat_null.npz')
low_Nsat_total = low_data_Nsat['Nsat']
low_Nsat35 = low_data_Nsat['low_Nsat35']

print(low_Nsat_total.shape)
print(low_Nsat35.shape)

# midz
mid_data_Nsat = np.load('/Users/mtownsend/Documents/LRG_Data/mid_Nsat_null.npz')
mid_Nsat_total = mid_data_Nsat['Nsat']
mid_Nsat35 = mid_data_Nsat['mid_nullNsat35']
mid_Nsat05 = mid_data_Nsat['mid_nullNsat05']

print(mid_Nsat_total.shape)
print(mid_Nsat35.shape)
print(mid_Nsat05.shape)

# highz
high_data_Nsat = np.load('/Users/mtownsend/Documents/LRG_Data/high_Nsat_null.npz')
high_Nsat_total = high_data_Nsat['Nsat']
high_Nsat35 = high_data_Nsat['high_nullNsat35']
high_Nsat05 = high_data_Nsat['high_nullNsat05']
high_Nsat65 = high_data_Nsat['high_nullNsat65']

print(high_Nsat_total.shape)
print(high_Nsat35.shape)
print(high_Nsat05.shape)
print(high_Nsat65.shape)



(5000, 49, 49, 49)
(5000, 49, 49, 49)
(5000, 49, 49, 49)
(1660, 49, 49, 49)
(3340, 49, 49, 49)
(5000, 49, 49, 49)
(836, 49, 49, 49)
(1667, 49, 49, 49)
(2497, 49, 49, 49)


In [4]:
# read in selection matrices

# lowz
smatrix_low = np.load('/Users/mtownsend/Desktop/lowz_smatrix-1matrices-v2.npz')
low_smatrix = smatrix_low['smatrix']
smatrix_low35 = low_smatrix[0]
print(smatrix_low35.shape)

print()
print()

# midz
smatrix_mid = np.load('/Users/mtownsend/Desktop/midz_smatrix-2matrices-v2.npz')
mid_smatrix = smatrix_mid['smatrix']
smatrix_mid35 = mid_smatrix[0]
smatrix_mid05 = mid_smatrix[1]
print(smatrix_mid35.shape)
print(smatrix_mid05.shape)

print()
print()

# highz
smatrix_high = np.load('/Users/mtownsend/Desktop/highz_smatrix-3matrices-v2.npz')
high_smatrix = smatrix_high['smatrix']
smatrix_high35 = high_smatrix[0]
smatrix_high05 = high_smatrix[1]
smatrix_high65 = high_smatrix[2]
print(smatrix_high35.shape)
print(smatrix_high05.shape)
print(smatrix_high65.shape)

(49, 49, 49)


(49, 49, 49)
(49, 49, 49)


(49, 49, 49)
(49, 49, 49)
(49, 49, 49)


In [5]:
# apply masks

# lowz
masked_Nsat_low35 = []
for i in range(len(low_Nsat35)):
    masked_Nsat_low35.append(low_Nsat35[i] * smatrix_low35)

print(np.asarray(masked_Nsat_low35).shape)

# midz
masked_Nsat_mid35 = []
for i in range(len(mid_Nsat35)):
    masked_Nsat_mid35.append(mid_Nsat35[i] * smatrix_mid35)
    
print(np.asarray(masked_Nsat_mid35).shape)
    
masked_Nsat_mid05 = []
for i in range(len(mid_Nsat05)):
    masked_Nsat_mid05.append(mid_Nsat05[i] * smatrix_mid05)
    
print(np.asarray(masked_Nsat_mid05).shape)


# highz
masked_Nsat_high35 = []
for i in range(len(high_Nsat35)):
    masked_Nsat_high35.append(high_Nsat35[i] * smatrix_high35)
    
print(np.asarray(masked_Nsat_high35).shape)
    
masked_Nsat_high05 = []
for i in range(len(high_Nsat05)):
    masked_Nsat_high05.append(high_Nsat05[i] * smatrix_high05)
    
print(np.asarray(masked_Nsat_high05).shape)
    
masked_Nsat_high65 = []
for i in range(len(high_Nsat65)):
    masked_Nsat_high65.append(high_Nsat65[i] * smatrix_high65)
    
print(np.asarray(masked_Nsat_high65).shape)
    

(5000, 49, 49, 49)
(1660, 49, 49, 49)
(3340, 49, 49, 49)
(836, 49, 49, 49)
(1667, 49, 49, 49)
(2497, 49, 49, 49)


In [15]:
def totalNsat(Nsat, lum, z1, z2):

    import matplotlib.pylab as plt 
    import numpy as np
    from scipy import stats
    
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height


    sumsat = []

    # Sum up number of satellite galaxies for every LRG
    for i in range(len(Nsat)):
        sumsat.append(np.sum(Nsat[i]))

    meansat = np.mean(sumsat)
    print("mean number of satellites is", meansat)
    
    mediansat = np.median(sumsat)
    print("median number of satellites is", mediansat)

    sdsat = np.std(sumsat)
#     print("standard deviation of satellites is", sdsat)

    sterr = stats.sem(sumsat)
#     print("standard error is", sterr)
    
    eighty = np.percentile(sumsat, 80.)
    print("80th percentile = ", eighty)
    ninety = np.percentile(sumsat, 90.)
    print("90th percentile = ", ninety)
    ninetynine = np.percentile(sumsat, 99.)
    print("99th percentile = ", ninetynine)
    

#     plt.rcParams["figure.figsize"] = [15, 8]
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
# #     plt.title("Histogram of the Number of Satellite Galaxies; log(lum)=10.36; 0.55 < z < 0.6", fontsize=25)
#     plt.title("Histogram of the Number of Satellite Galaxies; log(lum)={}; {} < z < {}".format(lum, z1, z2), fontsize=20)
#     plt.hist(sumsat, bins=50)
#     plt.xlabel(r'$Number$ $of$ $Satellite$ $Galaxies$', fontsize=25)
#     plt.ylabel(r'$counts$', fontsize=25)
#     plt.axvline(x=eighty, linewidth=1, ls='-', color='r')
#     plt.axvline(x=ninety, linewidth=1, ls='--', color='g')
#     plt.axvline(x=ninetynine, linewidth=1, ls=':', color='b')
#     plt.text(top, right,'80th = {}'.format(eighty), fontsize=14, ha='center', va='center', transform=ax.transAxes)
#     plt.text(top, right-0.05,'90th = {}'.format(ninety), fontsize=14, ha='center', va='center', transform=ax.transAxes)
#     plt.text(top, right-0.1,'99th = {}'.format(ninetynine), fontsize=14, ha='center', va='center', transform=ax.transAxes)
    # plt.show()
    return(eighty, ninety, ninetynine)


In [18]:
zrange = [0.2, 0.35, 0.5, 0.65]

# low
low_p35 = totalNsat(masked_Nsat_low35, 9.7, round(zrange[0], 2), round(zrange[1], 2))

np.savez('/Users/mtownsend/Desktop/MeetingPlots/08Jan2021/percentiles_low.npz', low_p35=low_p35)
data_low = np.load('/Users/mtownsend/Desktop/MeetingPlots/08Jan2021/percentiles_low.npz')
p35 = data['low_p35']
print(p35)

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat25_5000.pdf')
# plt.show()

print('----')

# mid
mid_p35 = totalNsat(masked_Nsat_mid35, 10.11, round(zrange[0], 2), round(zrange[1], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

mid_p05 = totalNsat(masked_Nsat_mid05, 10.11, round(zrange[1], 2), round(zrange[2], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

np.savez('/Users/mtownsend/Desktop/MeetingPlots/08Jan2021/percentiles_mid.npz', mid_p35=mid_p35, mid_p05=mid_p05)
data_mid = np.load('/Users/mtownsend/Desktop/MeetingPlots/08Jan2021/percentiles_mid.npz')
p35 = data_mid['mid_p35']
p05 = data_mid['mid_p05']
print(p35)
print(p05)

print('----')

# high
high_p35 = totalNsat(masked_Nsat_high35, 10.44, round(zrange[0], 2), round(zrange[1], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

high_p05 = totalNsat(masked_Nsat_high05, 10.44, round(zrange[1], 2), round(zrange[2], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

high_p65 = totalNsat(masked_Nsat_high65, 10.44, round(zrange[2], 2), round(zrange[3], 2))

# plt.savefig('/Users/mtownsend/Desktop/MeetingPlots/2Oct2020/lowz_random_maskedNsat03_5000.pdf')
# plt.show()

np.savez('/Users/mtownsend/Desktop/MeetingPlots/08Jan2021/percentiles_high.npz', high_p35=high_p35, high_p05=high_p05, high_p65=high_p65)
data_high = np.load('/Users/mtownsend/Desktop/MeetingPlots/08Jan2021/percentiles_high.npz')
p35 = data_high['high_p35']
p05 = data_high['high_p05']
p65 = data_high['high_p65']

print(p35)
print(p05)
print(p65)


mean number of satellites is 0.3713523302377361
median number of satellites is -0.007246479730776578
80th percentile =  1.651692686742171
90th percentile =  2.8035152759149105
99th percentile =  6.964422029096868
[1.65169269 2.80351528 6.96442203]
----
mean number of satellites is 0.08616970643440376
median number of satellites is -0.26626173680757637
80th percentile =  0.6448196103930999
90th percentile =  1.0578320408416582
99th percentile =  2.7077051132042533
mean number of satellites is 0.22492223729243274
median number of satellites is -0.059873497202842296
80th percentile =  1.1449253071588772
90th percentile =  2.0287015547429696
99th percentile =  5.400129503459292
[0.64481961 1.05783204 2.70770511]
[1.14492531 2.02870155 5.4001295 ]
----
mean number of satellites is 0.024344976981490213
median number of satellites is -0.09133873081204838
80th percentile =  -0.04763884853252781
90th percentile =  0.8436573300748406
99th percentile =  1.8480629176228887
mean number of satellite